In [21]:
import requests

import pandas as pd
from PIL import Image
import io
import numpy as np
np.random.seed(0)

def split_np_array_train_test(arr, train_test_ratio = 0.75):
    split = int(round(len(arr)*0.75,0))
    np.random.shuffle(arr)
    return (arr[:split],arr[split:],)


plankton_names = [
        "aphanizomenon",
        "asplanchna",
        "asterionella",
        "bosmina",
        "brachionus",
        "ceratium",
        "chaoborus",
        "conochilus",
        "copepod_skins",
        "cyclops",
        "daphnia",
        "daphnia_skins",
        "diaphanosoma",
        "diatom_chain",
        "dinobryon",
        "dirt",
        "eudiaptomus",
        "filament",
        "fish",
        "fragilaria",
        "hydra",
        "kellicottia",
        "keratella_cochlearis",
        "keratella_quadrata",
        "leptodora",
        "maybe_cyano",
        "nauplius",
        "paradileptus",
        "polyarthra",
        "rotifers",
        "synchaeta",
        "trichocerca",
        "unknown",
        "unknown_plankton",
        "uroglena",
    ]
def retrieve_images():
    repo_url_base = (
        "https://github.com/juleshenry/quantum-mnist/blob/main/data/zooplankton_0p5x/"
    )
    

    photo_prefix = "SPC-EAWAG-0P5X-"
    file_suffix = ".jpeg"
    raw_suffix = "?raw=True"
    data_dict = {}
    for plankton in plankton_names:
        repo_data = repo_url_base + plankton + "/training_data/"
        print("Retrieving..." + repo_data)
        r = requests.get(repo_data)
        for o in str(r.content).split(photo_prefix)[1:12]:
            img_url = (
                repo_data
                + photo_prefix
                + o.split(file_suffix)[0]
                + file_suffix
                + raw_suffix
            )
            print("Image@", img_url)
            img = requests.get(img_url)
            x = Image.open(io.BytesIO(img.content))
            x = x.convert("L")
            x = np.asarray(x)#[..., np.newaxis] / 255.0
            if data_dict.get(plankton):
                data_dict[plankton].append(x)
            else:
                data_dict[plankton] = [x]
        break
    return data_dict


import math
def bilinear_interpolation(image, y, x):
    height = image.shape[0]
    width = image.shape[1]

    x1 = max(min(math.floor(x), width - 1), 0)
    y1 = max(min(math.floor(y), height - 1), 0)
    x2 = max(min(math.ceil(x), width - 1), 0)
    y2 = max(min(math.ceil(y), height - 1), 0)

    a = float(image[y1, x1])
    b = float(image[y2, x1])
    c = float(image[y1, x2])
    d = float(image[y2, x2])

    dx = x - x1
    dy = y - y1

    new_pixel = a * (1 - dx) * (1 - dy)
    new_pixel += b * dy * (1 - dx)
    new_pixel += c * dx * (1 - dy)
    new_pixel += d * dx * dy
    return round(new_pixel)


def bl_resize(image, new_height, new_width):
    new_image = np.zeros((new_height, new_width), image.dtype)  # new_image = [[0 for _ in range(new_width)] for _ in range(new_height)]

    orig_height = image.shape[0]
    orig_width = image.shape[1]

    # Compute center column and center row
    x_orig_center = (orig_width-1) / 2
    y_orig_center = (orig_height-1) / 2

    # Compute center of resized image
    x_scaled_center = (new_width-1) / 2
    y_scaled_center = (new_height-1) / 2

    # Compute the scale in both axes
    scale_x = orig_width / new_width;
    scale_y = orig_height / new_height;

    for y in range(new_height):
        for x in range(new_width):
            x_ = (x - x_scaled_center) * scale_x + x_orig_center
            y_ = (y - y_scaled_center) * scale_y + y_orig_center

            new_image[y, x] = bilinear_interpolation(image, y_, x_)

    return new_image

if __name__ == "__main__":
    import time
    s = time.time()
    dd = retrieve_images()
    qubit_dims = (4,4,)
    for p in plankton_names:
        x_train, x_test = split_np_array_train_test(np.array(dd[p]))
        y_train, y_test = np.array([p]*len(x_train)), np.array([p]*len(x_test))
        x_train_small, x_test_small = [bl_resize(x,*qubit_dims) for x in x_train], [bl_resize(x,*qubit_dims) for x in x_test]
        o = Image.fromarray(x_train[0])
        o.show()
        print(x_train_small)
        o = Image.fromarray(x_train_small[0])
        o.show()

        break
    print(time.time() - s)

Retrieving...https://github.com/juleshenry/quantum-mnist/blob/main/data/zooplankton_0p5x/aphanizomenon/training_data/
Image@ https://github.com/juleshenry/quantum-mnist/blob/main/data/zooplankton_0p5x/aphanizomenon/training_data/SPC-EAWAG-0P5X-1570543372901157-3725350526242-001629-055-1224-2176-84-64.jpeg?raw=True
Image@ https://github.com/juleshenry/quantum-mnist/blob/main/data/zooplankton_0p5x/aphanizomenon/training_data/SPC-EAWAG-0P5X-1570543372901157-3725350526242-001629-055-1224-2176-84-64.jpeg?raw=True
Image@ https://github.com/juleshenry/quantum-mnist/blob/main/data/zooplankton_0p5x/aphanizomenon/training_data/SPC-EAWAG-0P5X-1570543372901157-3725350526242-001629-055-1224-2176-84-64.jpeg?raw=True
Image@ https://github.com/juleshenry/quantum-mnist/blob/main/data/zooplankton_0p5x/aphanizomenon/training_data/SPC-EAWAG-0P5X-1570543374882008-3725352526408-001649-138-1826-1500-72-56.jpeg?raw=True
Image@ https://github.com/juleshenry/quantum-mnist/blob/main/data/zooplankton_0p5x/aphaniz

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:145: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [ ]:
# 1. Load the data
## -In this notebook we will:
## 1. Load the plankton data as graysale images 
## 2. Run round robin comparisons with classical 4x4 simple FFN
## 3. Run round robin comparisons with quantum 4x4 example